In [29]:
import numpy as np
import pandas as pd
import sys
import os
from codebase.file_utils import save_obj, load_obj
from codebase.post_process import samples_to_df, get_topn_eig
import altair as alt
alt.data_transformers.disable_max_rows()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
log_dir = "./log/20200417_161857_efadeb_s1m5/"
data = load_obj('data', log_dir)
ps = load_obj('ps', log_dir)
ps.keys()


odict_keys(['beta', 'alpha', 'zz', 'yy'])

In [22]:
data

{'D': array([[0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 1, 1],
        ...,
        [0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 0]]), 'N': 565, 'J': 6, 'K': 2}

In [23]:
print(open(log_dir+'model.txt').read())

data {
  int<lower=1> N;
  int<lower=1> K;
  int<lower=1> J;
  int<lower=0, upper=1> DD[N, J];
}

parameters {
  vector[J] alpha;
  matrix[J,K] beta;
  matrix[N,K] zz;
}

transformed parameters{
  matrix[N,J] yy;
  for (n in 1:N) yy[n,] = to_row_vector(alpha) + zz[n,] * beta';
}
  
model {
  to_vector(beta) ~ normal(0, 1);
  to_vector(alpha) ~ normal(0, 10);
  to_vector(zz) ~ normal(0, 1);
  for (j in 1:J) DD[, j] ~ bernoulli_logit(yy[, j]);
}



In [25]:
num_chains = ps['alpha'].shape[1]
num_samples = ps['alpha'].shape[0]

ps['beta_rot'] = np.empty_like(ps['beta'])

for chain_number in range(ps['alpha'].shape[1]):
    if ps['beta'].ndim > 3:
        for i in range(num_samples):
            b = ps['beta'][i,chain_number]
            ps['beta_rot'][i,chain_number] = get_topn_eig(b@b.T, 2)['P']

In [27]:
samples_to_df(ps, 'beta_rot').head()

,index,val,param_name,cn,row,col
0,0,0.471629,beta_rot,0,0,0
1,1,0.373208,beta_rot,0,0,0
2,2,0.443896,beta_rot,0,0,0
3,3,0.467175,beta_rot,0,0,0
4,4,0.377989,beta_rot,0,0,0


In [28]:
c1 = alt.Chart(samples_to_df(ps, 'beta_rot')).mark_line(opacity=0.8).encode(
    alt.X('index:O'),
    alt.Y('val'),
    alt.Color('cn:N'),
    row = 'row:O',
    column= 'col:O'
).properties(width = 200, height=100, title='Beta') 

c1.interactive()

alt.Chart(...)

In [15]:
c1 = alt.Chart(samples_to_df(ps, 'alpha')).mark_line(opacity=0.8).encode(
    alt.X('index:O'),
    alt.Y('val'),
    alt.Color('cn:N'),
    row = 'row:O',
    column= 'cn:N'
).properties(width = 200, height=100) 

c1.interactive()

alt.Chart(...)